In [48]:
import psycopg2
import pandas as pd
import numpy as np
import mariadb
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
import pickle
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

In [18]:
water_df = pd.read_csv("output/water_analysis_cleaned.csv")

In [19]:
# water_df = water_df.loc[water_df['analysis_name'] == 'Base Titanium Water Analysis']
water_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,230,230,CO006WA0079,2012-07-19 00:00:00.000,Irrigation,sar,1.570,NaN
1,312,312,CP052WA0005,2012-07-13 00:00:00.000,Irrigation,sar,1.090,NaN
2,320,320,CP052WA0006,2012-07-13 00:00:00.000,Irrigation,sar,0.050,NaN
3,335,335,CP052WA0007,2012-07-13 00:00:00.000,Irrigation,sar,1.280,NaN
4,357,357,CP052WA0008,2012-07-13 00:00:00.000,Irrigation,sar,1.730,NaN
...,...,...,...,...,...,...,...,...
1006750,446831,446831,CU014WA0017,2020-04-09 10:34:00.000,KS Drinking Water Standard Analysis,nitrite,0.005,ppm
1006751,447540,447540,CU014WA0019,2020-04-09 11:01:00.000,KS Drinking Water Standard Analysis,nitrite,0.005,ppm
1006752,476832,476832,CB210WA0001,2020-05-12 10:49:00.000,KS Drinking Water Standard Analysis,nitrite,0.003,ppm
1006753,477818,477818,CB210WA0002,2020-05-12 10:49:00.000,KS Drinking Water Standard Analysis,nitrite,0.003,ppm


In [20]:
water_df.sort_values("batch_date")

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
739321,22111,22111,CM003WA0024,2010-08-25 00:00:00.000,Standard Drinking Water Analysis,fluorides,0.30,ppm
377993,22108,22108,CM003WA0024,2010-08-25 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,boron,0.01,ppm
464223,22125,22125,CM003WA0024,2010-08-25 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,manganese,0.05,ppm
729643,22117,22117,CM003WA0024,2010-08-25 00:00:00.000,Standard Drinking Water Analysis,calcium,9.42,ppm
488615,22116,22116,CM003WA0024,2010-08-25 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,ammonium,0.01,ppm
...,...,...,...,...,...,...,...,...
932687,392769,392769,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),silicon__,33.40,ppm
935714,398670,398670,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),total_coliforms,70.00,mpn/100ml
931118,392966,392966,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),ammonium,0.21,ppm
928797,392970,392970,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),sodium,150.00,ppm


In [21]:
water_df = water_df.sort_values('sample_code')

In [22]:
new_samples_df = water_df.loc[water_df['batch_date'] > '2024-05-21'] 
water_df = water_df.loc[water_df['batch_date'] <= '2024-05-21'] 

In [23]:
water_df.analysis_name.unique()

array(['Irrigation', 'Reverse Osmosis Water Use',
       'Feacal Coliforms (MPN) and E.Coli', 'Standard Drinking Water',
       'Standard Drinking Water Analysis (WHO)',
       'Total Coliform and Faecal E.Coli', 'Heavy Metal Analysis (Water)',
       'Standard Drinking Water Analysis',
       'Basic Drinking Water Analysis', 'Heavy Metals in Water',
       'Total Dissolved Solids', 'Water Colour Analysis', 'Turbidity',
       'Total Suspended Solids', 'Basic Drinking Water Analysis (WHO)',
       'Feacal Coliform (MPN) and E.Coli', 'Post Harvest Water Analysis',
       'Total Bacterial Count', 'Total Coliforms and Faecal E.Coli',
       'Total Bacteria Count', 'Total Suspended Solids (Calculated)',
       'KEBS Drinking Water Microbiology', 'Chemical Oxygen Demand',
       'Chemical & Biochemical Oxygen Demand', 'Water Borne Diseases',
       'E coli & Total Coliforms', 'Water Microbiology (KEBS)',
       'KEBS Drinking Water Standard Analysis',
       'Colloidal Silica in Water', 'Ba

In [24]:
water_df['analysis_name'].nunique()

39

In [25]:
new_samples_df['analysis_name'].nunique()

6

In [31]:
new_samples_df[(new_samples_df['analysis_name'].isin(water_df['analysis_name'].unique()))].analysis_name.nunique()

6

In [32]:
water_df['analysis_name'] = [i.replace(":","") for i in water_df['analysis_name']]

In [33]:
water_df['analysis_name'].value_counts()

Irrigation                                          374744
Feacal Coliforms (MPN) and E.Coli                   136524
Basic Drinking Water Analysis                       121173
Standard Drinking Water Analysis                    111316
Standard Drinking Water                              63749
Reverse Osmosis Water Use                            44505
Base Titanium Water Analysis                         35185
Total Coliform and Faecal E.Coli                     28720
Standard Drinking Water Analysis (WHO)               19280
Heavy Metal Analysis (Water)                         16642
Total Dissolved Solids                               11430
Taita Water Analysis (Uni of Helsinki)                8256
Water Colour Analysis                                 7449
Heavy Metals in Water                                 6250
Feacal Coliform (MPN) and E.Coli                      4776
Total Suspended Solids (Calculated)                   2871
Turbidity                                             28

In [46]:
for analysis in np.unique(water_df['analysis_name']):
    print(analysis)
    df_ = water_df.loc[water_df['analysis_name']==analysis]
    df_ = pd.pivot_table(data=df_, values="result", index="sample_code", columns="chemical_name")
    print(len(df_))

    os.makedirs(f"output/boxplots/{analysis}",exist_ok=True)
    for column in df_.columns:
        plt.boxplot(df_[column])
        plt.savefig(f"output/boxplots/{analysis}/{column}.png")
        plt.clf()
        outlier_threshold = df_[column].quantile(0.99)
        # df_ = df_.loc[df_[column] <= outlier_threshold]
    df_.describe().to_csv(f"output/chemical_null_count/{analysis}.csv")
    print(len(df_))
    if(len(df_) == 0):
        continue
    
    df_.to_csv(f"output/analysis/{analysis}.csv")
    

Aquasearch Full Chemical Analysis
191
191
BT Extended Water Quality Analysis
28
28
Base Titanium Water Analysis
1548
1548
Basic Drinking Water Analysis
2720
2720
Basic Drinking Water Analysis (WHO)
338
338
Chemical & Biochemical Oxygen Demand
160
160
Chemical Oxygen Demand
79
79
Colloidal Silica in Water
30
30
E coli & Total Coliforms
105
105
Feacal Coliform (MPN) and E.Coli
214
214
Feacal Coliforms (MPN) and E.Coli
3344
3344
Free Chlorine
77
77
Heavy Metal Analysis (Water)
624
624
Heavy Metals in Water
654
654
Irrigation
6632
6632
KEBS Drinking Water Microbiology
71
71
KEBS Drinking Water Standard Analysis
94
94
KS Drinking Water Standard Analysis
19
19
Oil & Grease
63
63
Post Harvest Water Analysis
121
121
Reverse Osmosis Water Use
1785
1785
Standard Drinking Water
1420
1420
Standard Drinking Water + Free Chlorine Analysis
43
43
Standard Drinking Water Analysis
2473
2473
Standard Drinking Water Analysis (WHO)
907
907
Taita Water Analysis (Uni of Helsinki)
516
516
Total Bacteria Count

<Figure size 640x480 with 0 Axes>

In [37]:
df_

chemical_name,total_viable_count_@37
sample_code,
CA361WA0008,5.0
CA361WA0009,130.0
CA361WA0010,5400.0
CA361WA0011,10000.0
CA361WA0012,3700.0
CA361WA0013,310.0
CB050WA0037,36.0
CB050WA0038,420.0
CB050WA0039,770.0


In [125]:
for analysis_file in os.listdir("output/analysis"):
    print(analysis_file)
    analysis_df = pd.read_csv(f"output/analysis/{analysis_file}",index_col=0)
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    analysis_df = pd.DataFrame(imp_mean.fit_transform(analysis_df), columns=analysis_df.columns, index=analysis_df.index)
    pickle.dump(imp_mean, open(f"models/imputers/{analysis_file.replace('csv','pkl')}","wb"))
    # if(len(analysis_df) == 0):
    #     continue
    sc = StandardScaler()
    pca = PCA()
    analysis_scaled = sc.fit_transform(analysis_df)
    pca.fit(analysis_scaled)
    pca_explained_variance_df = pd.DataFrame(zip(analysis_df.columns,pca.explained_variance_ratio_))
    pca_explained_variance_df.to_csv(f"output/pca_explained_variance_ratio/{analysis_file}")
    pca_explained_variance_df = pca_explained_variance_df.loc[pca_explained_variance_df[1]>0.1]
    n_components = len(pca_explained_variance_df)
    pca_chems = pca_explained_variance_df[0]
    pca = PCA(n_components=n_components)
    pca_reduced_df = pca.fit_transform(analysis_scaled)
    analysis_scaled = pd.DataFrame(analysis_scaled,index=analysis_df.index)
    pca_reduced_df = pd.DataFrame(pca_reduced_df,index=analysis_df.index, columns=pca_chems)
    analysis_scaled.to_csv(f"output/analysis_scaled/{analysis_file}")
    pca_reduced_df.to_csv(f"output/pca/{analysis_file}")
    pickle.dump(sc, open(f"models/scalers/{analysis_file.replace('csv','pkl')}","wb"))
    pickle.dump(pca, open(f"models/pca/{analysis_file.replace('csv','pkl')}","wb"))

Aquasearch Full Chemical Analysis.csv
Base Titanium Water Analysis.csv
Basic Drinking Water Analysis (WHO).csv
Basic Drinking Water Analysis.csv
BT Extended Water Quality Analysis.csv
Chemical & Biochemical Oxygen Demand.csv
Chemical Oxygen Demand.csv
Colloidal Silica in Water.csv
E coli & Total Coliforms.csv
Feacal Coliform (MPN) and E.Coli.csv
Feacal Coliforms (MPN) and E.Coli.csv
Free Chlorine.csv
Heavy Metal Analysis (Water).csv
Heavy Metals in Water.csv
Irrigation.csv
KEBS Drinking Water Microbiology.csv
KEBS Drinking Water Standard Analysis.csv
KS Drinking Water Standard Analysis.csv
Oil & Grease.csv
Post Harvest Water Analysis.csv
Reverse Osmosis Water Use.csv
Standard Drinking Water + Free Chlorine Analysis.csv
Standard Drinking Water Analysis (WHO).csv
Standard Drinking Water Analysis.csv
Standard Drinking Water.csv
Taita Water Analysis (Uni of Helsinki).csv
Total Bacteria Count.csv
Total Bacterial Count.csv
Total Chlorine.csv
Total Coliform and Faecal E.Coli.csv
Total Colifor

In [54]:
# pd.DataFrame([
#     {
#         'sample_code': 'CB057WA0107',
#         'ammonium': {'result':3.936512250255515, 'units':'ppm'},
#         'bicarbonate': {'result':3.936512250255515, 'units':'ppm'},
#         'boron': {'result':3.936512250255515, 'units':'ppm'}
#     }
# ])

In [55]:
# pca_reduced_df.reset_index().to_dict('records')

In [56]:
for analysis_file in os.listdir("output/pca"):
    print(analysis_file)

Aquasearch Full Chemical Analysis.csv
Base Titanium Water Analysis.csv
Basic Drinking Water Analysis (WHO).csv
Basic Drinking Water Analysis.csv
BT Extended Water Quality Analysis.csv
Chemical & Biochemical Oxygen Demand.csv
Chemical Oxygen Demand.csv
Colloidal Silica in Water.csv
E coli & Total Coliforms.csv
Feacal Coliform (MPN) and E.Coli.csv
Feacal Coliforms (MPN) and E.Coli.csv
Free Chlorine.csv
Heavy Metal Analysis (Water).csv
Heavy Metals in Water.csv
Irrigation.csv
KEBS Drinking Water Microbiology.csv
KEBS Drinking Water Standard Analysis.csv
KS Drinking Water Standard Analysis.csv
Oil & Grease.csv
Post Harvest Water Analysis.csv
Reverse Osmosis Water Use.csv
Standard Drinking Water + Free Chlorine Analysis.csv
Standard Drinking Water Analysis (WHO).csv
Standard Drinking Water Analysis.csv
Standard Drinking Water.csv
Taita Water Analysis (Uni of Helsinki).csv
Total Bacteria Count.csv
Total Bacterial Count.csv
Total Chlorine.csv
Total Coliform and Faecal E.Coli.csv
Total Colifor

In [57]:
for analysis_file in os.listdir("output/pca"):
    print(analysis_file)
    pca_reduced_df = pd.read_csv(f"output/pca/{analysis_file}",index_col=0)
    # if len(pca_reduced_df.columns) < 2:
        # continue
    mu = np.mean(pca_reduced_df, axis=0)
    sigma = np.cov(pca_reduced_df.T)

    try:
        pca_reduced_df['mahalanobis_distance'] = [distance.mahalanobis(pca_reduced_df.iloc[i], mu, np.linalg.inv(sigma)) for i in range(len(pca_reduced_df)) ]
    except:
        continue
    pca_reduced_df.to_csv(f"output/mahalanobis_distance/{analysis_file}")

Aquasearch Full Chemical Analysis.csv
Base Titanium Water Analysis.csv
Basic Drinking Water Analysis (WHO).csv
Basic Drinking Water Analysis.csv
BT Extended Water Quality Analysis.csv
Chemical & Biochemical Oxygen Demand.csv
Chemical Oxygen Demand.csv
Colloidal Silica in Water.csv
E coli & Total Coliforms.csv
Feacal Coliform (MPN) and E.Coli.csv
Feacal Coliforms (MPN) and E.Coli.csv
Free Chlorine.csv
Heavy Metal Analysis (Water).csv
Heavy Metals in Water.csv
Irrigation.csv
KEBS Drinking Water Microbiology.csv
KEBS Drinking Water Standard Analysis.csv
KS Drinking Water Standard Analysis.csv
Oil & Grease.csv
Post Harvest Water Analysis.csv
Reverse Osmosis Water Use.csv
Standard Drinking Water + Free Chlorine Analysis.csv
Standard Drinking Water Analysis (WHO).csv
Standard Drinking Water Analysis.csv
Standard Drinking Water.csv
Taita Water Analysis (Uni of Helsinki).csv
Total Bacteria Count.csv
Total Bacterial Count.csv
Total Chlorine.csv
Total Coliform and Faecal E.Coli.csv
Total Colifor

In [58]:
pca_reduced_df

,total_viable_count_@37
sample_code,
CA361WA0008,-0.814164
CA361WA0009,-0.778229
CA361WA0010,0.736775
CA361WA0011,2.059170
CA361WA0012,0.248064
CA361WA0013,-0.726483
CB050WA0037,-0.805252
CB050WA0038,-0.694861
CB050WA0039,-0.594244


In [59]:
mahalanobis_threshold_dict = {}
for analysis_file in os.listdir("output/mahalanobis_distance"):
    print(analysis_file)
    analysis = analysis_file.replace(".csv","")
    mahalanobis_df = pd.read_csv(f"output/mahalanobis_distance/{analysis_file}",index_col=0)
    upper_quantile = (mahalanobis_df['mahalanobis_distance'].quantile(0.95))
    mahalanobis_threshold_dict[analysis_file.replace(".csv","")] = upper_quantile
    mahalanobis_df.loc[mahalanobis_df['mahalanobis_distance'] >= upper_quantile].to_csv(f"output/mahanobis_distance_upper_quantile/{analysis_file}")
    plt.boxplot(mahalanobis_df['mahalanobis_distance'])
    plt.savefig(f"output/mahalanobis_boxplots/{analysis}.png")
    plt.clf()
pickle.dump(mahalanobis_threshold_dict, open("mahalanobis_thresholds.dict","wb"))

Aquasearch Full Chemical Analysis.csv
Base Titanium Water Analysis.csv
Basic Drinking Water Analysis (WHO).csv
BT Extended Water Quality Analysis.csv
Chemical & Biochemical Oxygen Demand.csv
Chemical Oxygen Demand.csv
Colloidal Silica in Water.csv
E coli & Total Coliforms.csv
Feacal Coliform (MPN) and E.Coli.csv
Free Chlorine.csv
Heavy Metal Analysis (Water).csv
Heavy Metals in Water.csv
Irrigation.csv
KEBS Drinking Water Microbiology.csv
KEBS Drinking Water Standard Analysis.csv
KS Drinking Water Standard Analysis.csv
Oil & Grease.csv
Reverse Osmosis Water Use.csv
Standard Drinking Water + Free Chlorine Analysis.csv
Standard Drinking Water Analysis.csv
Standard Drinking Water.csv
Taita Water Analysis (Uni of Helsinki).csv
Total Bacteria Count.csv
Total Bacterial Count.csv
Total Chlorine.csv
Total Coliform and Faecal E.Coli.csv
Total Coliforms and Faecal E.Coli.csv
Total Suspended Solids (Calculated).csv
Total Suspended Solids.csv
Turbidity Analysis.csv
Turbidity.csv
Water Borne Diseas

<Figure size 640x480 with 0 Axes>

In [60]:
mahalanobis_threshold_dict

{'Aquasearch Full Chemical Analysis': 4.30256039486611,
 'Base Titanium Water Analysis': 2.281347570068289,
 'Basic Drinking Water Analysis (WHO)': 2.179297665026459,
 'BT Extended Water Quality Analysis': 2.7494155696048312,
 'Chemical & Biochemical Oxygen Demand': 1.4012820121218432,
 'Chemical Oxygen Demand': 4.099201656951722,
 'Colloidal Silica in Water': 2.778499787336087,
 'E coli & Total Coliforms': 3.0200093009331237,
 'Feacal Coliform (MPN) and E.Coli': 2.421053282025681,
 'Free Chlorine': 3.185184009681968,
 'Heavy Metal Analysis (Water)': 2.9664717695608775,
 'Heavy Metals in Water': 2.392906109923457,
 'Irrigation': 0.6091339547639765,
 'KEBS Drinking Water Microbiology': 4.812911710601007,
 'KEBS Drinking Water Standard Analysis': 3.6903320550585637,
 'KS Drinking Water Standard Analysis': 3.3443272528079038,
 'Oil & Grease': 6.18712401484095,
 'Reverse Osmosis Water Use': 1.3638241836282,
 'Standard Drinking Water + Free Chlorine Analysis': 1.8222064765648445,
 'Standard

In [63]:
new_samples_df.analysis_name.unique()

array(['Standard Drinking Water Analysis (WHO)',
       'KEBS Drinking Water Microbiology',
       'Basic Drinking Water Analysis (WHO)', 'Irrigation',
       'Standard Drinking Water + Free Chlorine Analysis',
       'Reverse Osmosis Water Use'], dtype=object)

In [78]:
water_ = new_samples_df.loc[new_samples_df['analysis_name'].isin(mahalanobis_threshold_dict.keys())]

In [79]:
water_

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
1005615,267276,267276,CA373WA0077,2024-06-21 14:10:00.000,KEBS Drinking Water Microbiology,electrical_conductivity,0.18,mS cm -1
980118,271966,271966,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),calcium,13.40,ppm
1005616,271964,271964,CD048WA0110,2024-06-21 12:41:00.000,KEBS Drinking Water Microbiology,electrical_conductivity,0.28,mS cm -1
980449,271967,271967,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),magnesium,3.90,ppm
106687,387408,387408,CE021WA0022,2024-05-21 12:38:28.263,Irrigation,silicon__,5.79,ppm
...,...,...,...,...,...,...,...,...
980124,278510,278510,CT426WA0005,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),calcium,20.80,ppm
980133,296876,296876,CT426WA0006,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),calcium,136.00,ppm
980466,296877,296877,CT426WA0006,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),magnesium,101.00,ppm
980467,296883,296883,CT426WA0007,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),magnesium,8.12,ppm


In [80]:
water_ = water_.sort_values(by="sample_code")

In [81]:
water_ = water_.loc[(water_['sample_code'] == 'CD048WA0110')]

In [82]:
water_

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
980118,271966,271966,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),calcium,13.40,ppm
1005616,271964,271964,CD048WA0110,2024-06-21 12:41:00.000,KEBS Drinking Water Microbiology,electrical_conductivity,0.28,mS cm -1
980449,271967,271967,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),magnesium,3.90,ppm


In [83]:
water_pivot = pd.pivot_table(data = water_, columns = "chemical_name", values = "result", index="sample_code").reset_index()

In [87]:
water_

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
980118,271966,271966,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),calcium,13.40,ppm
1005616,271964,271964,CD048WA0110,2024-06-21 12:41:00.000,KEBS Drinking Water Microbiology,electrical_conductivity,0.28,mS cm -1
980449,271967,271967,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),magnesium,3.90,ppm


In [84]:
water_pivot['analysis_id'] = 570

In [85]:
req = water_pivot.to_dict(orient="records")

In [86]:
req

[{'sample_code': 'CD048WA0110',
  'calcium': 13.4,
  'electrical_conductivity': 0.28,
  'magnesium': 3.9,
  'analysis_id': 570}]

In [89]:
import psycopg2

conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [153]:
analysis_dict = pd.read_sql(f"""SELECT analysis_id, analysis_name FROM Analysis WHERE analysis_name IN {tuple(mahalanobis_threshold_dict.keys())}  ORDER BY analysis_name""",con=conn_lims).set_index("analysis_id").to_dict()['analysis_name']


C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [154]:
analysis_dict

{555: 'Aquasearch Full Chemical Analysis',
 719: 'Base Titanium Water Analysis',
 288: 'Basic Drinking Water Analysis (WHO)',
 743: 'BT Extended Water Quality Analysis',
 268: 'Chemical & Biochemical Oxygen Demand',
 26953: 'Chemical & Biochemical Oxygen Demand',
 27110: 'Chemical Oxygen Demand',
 27378: 'Chemical Oxygen Demand',
 188: 'Chemical Oxygen Demand',
 720: 'Chemical Oxygen Demand',
 732: 'Chemical Oxygen Demand',
 678: 'Colloidal Silica in Water',
 263: 'E coli & Total Coliforms',
 289: 'E Coli & Total Coliforms',
 459: 'E coli & Total Coliforms',
 26552: 'E Coli & Total Coliforms',
 27149: 'E coli & Total Coliforms',
 27038: 'E coli & Total Coliforms',
 26970: 'E coli & Total Coliforms',
 26805: 'E coli & Total Coliforms',
 26902: 'Free Chlorine',
 26630: 'Free Chlorine',
 26685: 'Heavy Metals in Water',
 276: 'Heavy Metals in Water',
 26724: 'KEBS Drinking Water Microbiology',
 27032: 'KEBS Drinking Water Microbiology',
 26860: 'KEBS Drinking Water Standard Analysis',
 267

In [155]:
pickle.dump(analysis_dict, open("analysis.dict","wb"))

In [228]:
req = [
    {
      'sample_code': 'CD048WA0110',
      'calcium': {'result':13.4, "units":"ppm"},
      'electrical_conductivity': {"result":0.28, "units":"mS cm -1"},
      'magnesium': {'result':3.9, "units":"ppm"},
      'analysis_id': [288,27032]
    }]


In [229]:
req

[{'sample_code': 'CD048WA0110',
  'calcium': {'result': 13.4, 'units': 'ppm'},
  'electrical_conductivity': {'result': 0.28, 'units': 'mS cm -1'},
  'magnesium': {'result': 3.9, 'units': 'ppm'},
  'analysis_id': [288, 27032]}]

In [188]:
_ = pd.DataFrame(req)

In [189]:
_

,sample_code,calcium,electrical_conductivity,magnesium,analysis_id
0,CD048WA0110,"{'result': 13.4, 'units': 'ppm'}","{'result': 0.28, 'units': 'mS cm -1'}","{'result': 3.9, 'units': 'ppm'}","[288, 27032]"


In [170]:
analysis_dict = pickle.load(open("analysis.dict","rb"))

In [171]:
analysis_dict

{555: 'Aquasearch Full Chemical Analysis',
 719: 'Base Titanium Water Analysis',
 288: 'Basic Drinking Water Analysis (WHO)',
 743: 'BT Extended Water Quality Analysis',
 268: 'Chemical & Biochemical Oxygen Demand',
 26953: 'Chemical & Biochemical Oxygen Demand',
 27110: 'Chemical Oxygen Demand',
 27378: 'Chemical Oxygen Demand',
 188: 'Chemical Oxygen Demand',
 720: 'Chemical Oxygen Demand',
 732: 'Chemical Oxygen Demand',
 678: 'Colloidal Silica in Water',
 263: 'E coli & Total Coliforms',
 289: 'E Coli & Total Coliforms',
 459: 'E coli & Total Coliforms',
 26552: 'E Coli & Total Coliforms',
 27149: 'E coli & Total Coliforms',
 27038: 'E coli & Total Coliforms',
 26970: 'E coli & Total Coliforms',
 26805: 'E coli & Total Coliforms',
 26902: 'Free Chlorine',
 26630: 'Free Chlorine',
 26685: 'Heavy Metals in Water',
 276: 'Heavy Metals in Water',
 26724: 'KEBS Drinking Water Microbiology',
 27032: 'KEBS Drinking Water Microbiology',
 26860: 'KEBS Drinking Water Standard Analysis',
 267

In [172]:
mahalanobis_thresholds = pickle.load(open("mahalanobis_thresholds.dict","rb"))

In [173]:
mahalanobis_thresholds

{'Aquasearch Full Chemical Analysis': 4.30256039486611,
 'Base Titanium Water Analysis': 2.281347570068289,
 'Basic Drinking Water Analysis (WHO)': 2.179297665026459,
 'BT Extended Water Quality Analysis': 2.7494155696048312,
 'Chemical & Biochemical Oxygen Demand': 1.4012820121218432,
 'Chemical Oxygen Demand': 4.099201656951722,
 'Colloidal Silica in Water': 2.778499787336087,
 'E coli & Total Coliforms': 3.0200093009331237,
 'Feacal Coliform (MPN) and E.Coli': 2.421053282025681,
 'Free Chlorine': 3.185184009681968,
 'Heavy Metal Analysis (Water)': 2.9664717695608775,
 'Heavy Metals in Water': 2.392906109923457,
 'Irrigation': 0.6091339547639765,
 'KEBS Drinking Water Microbiology': 4.812911710601007,
 'KEBS Drinking Water Standard Analysis': 3.6903320550585637,
 'KS Drinking Water Standard Analysis': 3.3443272528079038,
 'Oil & Grease': 6.18712401484095,
 'Reverse Osmosis Water Use': 1.3638241836282,
 'Standard Drinking Water + Free Chlorine Analysis': 1.8222064765648445,
 'Standard

In [195]:
str(req)

"[{'sample_code': 'CD048WA0110', 'calcium': {'result': 13.4, 'units': 'ppm'}, 'electrical_conductivity': {'result': 0.28, 'units': 'mS cm -1'}, 'magnesium': {'result': 3.9, 'units': 'ppm'}, 'analysis_id': [288, 27032]}]"

In [190]:
unit_decision = pd.read_csv("output/water_unit_per_chemical_decision.csv")

In [191]:
unit_decision

,Unnamed: 0,crop,chemical_name,unit_name
0,0,Irrigation,sar,NaN
1,0,Irrigation,nitrate_n,ppm
2,0,Irrigation,potassium,ppm
3,0,Irrigation,sulphur,ppm
4,0,Irrigation,phosphorus,ppm
...,...,...,...,...
832,0,Colloidal Silica in Water,colloidal_silica,ppm
833,0,Water Microbiology (KEBS),total_viable_count_@37,mpn/ml
834,0,KS Drinking Water Standard Analysis,total_coliforms,cfu/ml
835,0,KS Drinking Water Standard Analysis,faecal_e_coli,cfu/ml


In [225]:
result = {}
_df = pd.DataFrame()
for index, row in _.iterrows():
    analyses = row['analysis_id']
    for analysis_ in analyses:
        df_analysis_ = pd.DataFrame(row).T
        df_analysis_['analysis_name'] = analysis_dict[analysis_]
        _df = pd.concat([_df, df_analysis_])
for index,row in _df.iterrows():
    print(index)
    
    sample_code = row['sample_code']
    analysis = row['analysis_name']
    analysis_id = row['analysis_id']
    if sample_code not in result.keys():
        result[sample_code] = []
    scaler = pickle.load(open(f"models/scalers/{analysis}.pkl","rb"))
    pca = pickle.load(open(f"models/pca/{analysis}.pkl","rb"))
    imputer = pickle.load(open(f"models/imputers/{analysis}.pkl","rb"))

    pca_df = pd.read_csv(f"api/pca_df/{analysis}.csv",index_col=0)

    
    try :
        tmp_df = pd.DataFrame(row).T[pca_df.columns]   
        print(tmp_df)
    except:
        result[sample_code].append({"sample_code": sample_code,"status":"fail", "message": f"Missing parameters for analysis_id: {analysis_id}", "details": f"Expected parameters are {','.join(pca_df.columns)} for analysis: {analysis}" })
        continue
    failed_units_comparison = {}    
    for col in pca_df.columns:
        expected_units = unit_decision.loc[(unit_decision['crop'] == analysis) & (unit_decision['chemical_name'] == col)]
        print(expected_units[['crop','chemical_name','unit_name']].to_dict())
        if row[col]['units'] !=   expected_units['unit_name'].values[0] :
            failed_units_comparison[col] = expected_units[['crop','chemical_name','unit_name']].to_dict()
        else:
            row[col] = row[col]['result']
    if len(failed_units_comparison.keys()) > 0:
        result[sample_code].append({"sample_code": sample_code,"status":"fail", "message": f"Wrong units provided", "details": f"Expected units are {str(failed_units_comparison)} for analysis: {analysis}" })
        continue

    tmp_df = pd.DataFrame(row).T[pca_df.columns]    
    
    # tmp_df = 
    df_scaled = scaler.transform(tmp_df)
    df_pca = pd.DataFrame(pca.transform(df_scaled))

    mu = np.mean(pca_df, axis=0)
    sigma = np.cov(pca_df.T)

    mahalanobis_distance = distance.mahalanobis(df_pca.iloc[0], mu, np.linalg.inv(sigma))

    print(mahalanobis_distance)

    expected_md = mahalanobis_thresholds[analysis]
    print(expected_md)

    if mahalanobis_distance > expected_md:
        result[sample_code].append({"sample_code": sample_code,"status":"fail", "message": "Mahalanobis distance exceeds threshold", "description":f"Mahalanobis distance of {mahalanobis_distance} exceeds threshold of {expected_md} for analysis: {analysis}" })
    else:
        result[sample_code].append({"sample_code": sample_code,"status":"pass","message": "Mahalanobis distance within threshold", "description":f"Mahalanobis distance of {mahalanobis_distance} is within threshold of {expected_md} for analysis: {analysis}" })
    

0
                            calcium                        magnesium
0  {'result': 13.4, 'units': 'ppm'}  {'result': 3.9, 'units': 'ppm'}
{'crop': {549: 'Basic Drinking Water Analysis (WHO)'}, 'chemical_name': {549: 'calcium'}, 'unit_name': {549: 'ppm'}}
{'crop': {550: 'Basic Drinking Water Analysis (WHO)'}, 'chemical_name': {550: 'magnesium'}, 'unit_name': {550: 'ppm'}}
0.3319501004886362
2.179297665026459
0


In [227]:
result

{'CD048WA0110': [{'sample_code': 'CD048WA0110',
   'status': 'pass',
   'message': 'Mahalanobis distance within threshold',
   'description': 'Mahalanobis distance of 0.3319501004886362 is within threshold of 2.179297665026459 for analysis: Basic Drinking Water Analysis (WHO)'},
  {'sample_code': 'CD048WA0110',
   'status': 'fail',
   'message': 'Missing parameters for analysis_id: [288, 27032]',
   'details': 'Expected parameters are e_coli,electrical_conductivity,strontium,total_coliforms for analysis: KEBS Drinking Water Microbiology'}]}

In [218]:
str([1,2,3,4])

'[1, 2, 3, 4]'